In [7]:
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

In [2]:
initial_values = Table.read('../spectrum_analysis/galah_dr4_initial_parameters_220714_lite.fits')

In [3]:
grids = Table.read('galah_dr4_model_trainingset_gridpoints.fits')

In [34]:
grids['model_name'] = np.array([str(int(grid['teff_subgrid']))+'_'+str("{:.2f}".format(grid['logg_subgrid']))+'_'+str("{:.2f}".format(grid['fe_h_subgrid'])) for grid in grids])

In [8]:
grids = Table.read('../spectrum_grids/galah_dr4_model_trainingset_gridpoints.fits')
grid_index_tree = cKDTree(np.c_[grids['teff_subgrid']/1000.,grids['logg_subgrid'],grids['fe_h_subgrid']])

In [24]:
initial_values['grid_index'] = np.array([grid_index_tree.query([tgf[0]/1000.,tgf[1],tgf[2]]) for tgf in np.array([initial_values['teff'],initial_values['logg'],initial_values['fe_h']]).T])[:,1]

In [64]:
indices = initial_values['grid_index']
priority = Table()
d,c = np.unique(indices, return_counts=True)
priority['grid_index'] = np.array(d,dtype=int)
priority['counts_dr4_init'] = np.array(c,dtype=int)
priority.sort(keys='counts_dr4_init',reverse=True)

In [65]:
grids_done = [
    
]

In [66]:
grids_sme_to_train = []

In [67]:
trained_6 = []

In [68]:
s = []
for grid_p in grids_done:
    if grid_p not in grids_sme_to_train:
        print(str(grid_p)+',')
        s.append(grid_p)
print('import os')
print('ps = ['+','.join([str(x) for x in s])+']')
print('for p in ps:')
print("    os.system('ipython galah_dr4_grid_interpolation_trainingset_creation.py '+str(p))")
print("os.system('ipython galah_dr4_grid_interpolation_recommend_labels.py '+str(p))")

import os
ps = []
for p in ps:
    os.system('ipython galah_dr4_grid_interpolation_trainingset_creation.py '+str(p))
os.system('ipython galah_dr4_grid_interpolation_recommend_labels.py '+str(p))


In [69]:
def get_progress():
    progress_table = Table()
    progress_table['index_subgrid'] = grids['index_subgrid']
    # Let's get placeholders for the important progress indicators: nr grids calculated, nr grids converted into training_data, training_fully finished
    progress_table['trainingset_done'] = np.zeros(len(grids['index_subgrid']),dtype=int)
    progress_table['grids_trainingset'] = np.zeros(len(grids['index_subgrid']),dtype=int)
    progress_table['grids_calculated'] = np.zeros(len(grids['index_subgrid']),dtype=int)
    progress_table['next_conversion'] = np.zeros(len(grids['index_subgrid']),dtype=int)
    progress_table['next_calculation'] = np.zeros(len(grids['index_subgrid']),dtype=int)
    # Convert all 7 grids to their gridnames
    progress_table['grid_1'] = np.array([str(int(grids['teff_subgrid'][x]))+'_'+"{:.2f}".format(grids['logg_subgrid'][x])+'_'+"{:.2f}".format(grids['fe_h_subgrid'][x]) for x in range(len(grids['index_subgrid']))])
    progress_table['grid_2'] = np.array([str(int(grids['teff_low_subgrid'][x]))+'_'+"{:.2f}".format(grids['logg_subgrid'][x])+'_'+"{:.2f}".format(grids['fe_h_subgrid'][x]) for x in range(len(grids['index_subgrid']))])
    progress_table['grid_3'] = np.array([str(int(grids['teff_high_subgrid'][x]))+'_'+"{:.2f}".format(grids['logg_subgrid'][x])+'_'+"{:.2f}".format(grids['fe_h_subgrid'][x]) for x in range(len(grids['index_subgrid']))])
    progress_table['grid_4'] = np.array([str(int(grids['teff_subgrid'][x]))+'_'+"{:.2f}".format(grids['logg_low_subgrid'][x])+'_'+"{:.2f}".format(grids['fe_h_subgrid'][x]) for x in range(len(grids['index_subgrid']))])
    progress_table['grid_5'] = np.array([str(int(grids['teff_subgrid'][x]))+'_'+"{:.2f}".format(grids['logg_high_subgrid'][x])+'_'+"{:.2f}".format(grids['fe_h_subgrid'][x]) for x in range(len(grids['index_subgrid']))])
    progress_table['grid_6'] = np.array([str(int(grids['teff_subgrid'][x]))+'_'+"{:.2f}".format(grids['logg_subgrid'][x])+'_'+"{:.2f}".format(grids['fe_h_low_subgrid'][x]) for x in range(len(grids['index_subgrid']))])
    progress_table['grid_7'] = np.array([str(int(grids['teff_subgrid'][x]))+'_'+"{:.2f}".format(grids['logg_subgrid'][x])+'_'+"{:.2f}".format(grids['fe_h_high_subgrid'][x]) for x in range(len(grids['index_subgrid']))])
    # Now let's set zeros for the progress to initiate
    progress_table['grid_1_calculated'] = np.zeros(len(grids['index_subgrid']))
    progress_table['grid_2_calculated'] = np.zeros(len(grids['index_subgrid']))
    progress_table['grid_3_calculated'] = np.zeros(len(grids['index_subgrid']))
    progress_table['grid_4_calculated'] = np.zeros(len(grids['index_subgrid']))
    progress_table['grid_5_calculated'] = np.zeros(len(grids['index_subgrid']))
    progress_table['grid_6_calculated'] = np.zeros(len(grids['index_subgrid']))
    progress_table['grid_7_calculated'] = np.zeros(len(grids['index_subgrid']))
    # Let's see which ones we have already put into a training set
    progress_table['grid_1_trainingset'] = np.zeros(len(grids['index_subgrid']))
    progress_table['grid_2_trainingset'] = np.zeros(len(grids['index_subgrid']))
    progress_table['grid_3_trainingset'] = np.zeros(len(grids['index_subgrid']))
    progress_table['grid_4_trainingset'] = np.zeros(len(grids['index_subgrid']))
    progress_table['grid_5_trainingset'] = np.zeros(len(grids['index_subgrid']))
    progress_table['grid_6_trainingset'] = np.zeros(len(grids['index_subgrid']))
    progress_table['grid_7_trainingset'] = np.zeros(len(grids['index_subgrid']))
    # Some grids cannot be calculated, let's set their value to 1 already
    # For example, we do not have grids with 5.0 logg for Teff > 4000
    progress_table['grid_5_calculated'][np.where((grids['teff_subgrid'] > 4000) & (grids['logg_high_subgrid'] > 4.75))[0]] = 1
    progress_table['grid_5_trainingset'][np.where((grids['teff_subgrid'] > 4000) & (grids['logg_high_subgrid'] > 4.75))[0]] = 1
    # Then go through the list of the ones that we actually have calculated
    for grid_done in grids_done:
        that_grid = progress_table['grid_1'][grid_done]
        try:
            progress_table['grid_1_calculated'][np.where(progress_table['grid_1'] == that_grid)[0][0]] = 1
        except:
            pass
        try:
            progress_table['grid_2_calculated'][np.where(progress_table['grid_2'] == that_grid)[0][0]] = 1
        except:
            pass
        try:
            progress_table['grid_3_calculated'][np.where(progress_table['grid_3'] == that_grid)[0][0]] = 1
        except:
            pass
        try:
            progress_table['grid_4_calculated'][np.where(progress_table['grid_4'] == that_grid)[0][0]] = 1
        except:
            pass
        try:
            progress_table['grid_5_calculated'][np.where(progress_table['grid_5'] == that_grid)[0][0]] = 1
        except:
            pass
        try:
            progress_table['grid_6_calculated'][np.where(progress_table['grid_6'] == that_grid)[0][0]] = 1
        except:
            pass
        try:
            progress_table['grid_7_calculated'][np.where(progress_table['grid_7'] == that_grid)[0][0]] = 1
        except:
            pass
    # Then go through the list of the ones that we actually have transformed into a training_data
    for grid_done in grids_sme_to_train:
        that_grid = progress_table['grid_1'][grid_done]
        try:
            progress_table['grid_1_trainingset'][np.where(progress_table['grid_1'] == that_grid)[0][0]] = 1
        except:
            pass
        try:
            progress_table['grid_2_trainingset'][np.where(progress_table['grid_2'] == that_grid)[0][0]] = 1
        except:
            pass
        try:
            progress_table['grid_3_trainingset'][np.where(progress_table['grid_3'] == that_grid)[0][0]] = 1
        except:
            pass
        try:
            progress_table['grid_4_trainingset'][np.where(progress_table['grid_4'] == that_grid)[0][0]] = 1
        except:
            pass
        try:
            progress_table['grid_5_trainingset'][np.where(progress_table['grid_5'] == that_grid)[0][0]] = 1
        except:
            pass
        try:
            progress_table['grid_6_trainingset'][np.where(progress_table['grid_6'] == that_grid)[0][0]] = 1
        except:
            pass
        try:
            progress_table['grid_7_trainingset'][np.where(progress_table['grid_7'] == that_grid)[0][0]] = 1
        except:
            pass
    progress_table['grids_calculated'] = np.sum(progress_table['grid_'+str(n)+'_calculated'] for n in [1,2,3,4,5,6,7])
    progress_table['grids_trainingset'] = np.sum(progress_table['grid_'+str(n)+'_trainingset'] for n in [1,2,3,4,5,6,7])
    for index in trained_6:
        progress_table['trainingset_done'][index] = 1
    for index in range(len(progress_table['grids_trainingset'])):
        if progress_table['grids_calculated'][index] == 6:
            missing_grid = np.where(np.array([progress_table['grid_'+str(n)+'_calculated'][index] for n in [1,2,3,4,5,6,7]])==0)[0][0]
            index_missing_grid = np.where(progress_table['grid_1'] == progress_table['grid_'+str(missing_grid+1)][index])[0]
            if len(index_missing_grid) > 0:
                index_missing_grid = index_missing_grid[0]
            else:
                index_missing_grid = -1
            progress_table['next_calculation'][index] = index_missing_grid
        if progress_table['grids_calculated'][index] == 5:
            missing_grid = np.where(np.array([progress_table['grid_'+str(n)+'_calculated'][index] for n in [1,2,3,4,5,6,7]])==0)[0][0]
            index_missing_grid = np.where(progress_table['grid_1'] == progress_table['grid_'+str(missing_grid+1)][index])[0]
            if len(index_missing_grid) > 0:
                index_missing_grid = index_missing_grid[0]
            else:
                index_missing_grid = -1
            progress_table['next_calculation'][index] = index_missing_grid
        if progress_table['grids_calculated'][index] == 4:
            missing_grid = np.where(np.array([progress_table['grid_'+str(n)+'_calculated'][index] for n in [1,2,3,4,5,6,7]])==0)[0][0]
            index_missing_grid = np.where(progress_table['grid_1'] == progress_table['grid_'+str(missing_grid+1)][index])[0]
            if len(index_missing_grid) > 0:
                index_missing_grid = index_missing_grid[0]
            else:
                index_missing_grid = -1
            progress_table['next_calculation'][index] = index_missing_grid
        if progress_table['grids_calculated'][index] == 3:
            missing_grid = np.where(np.array([progress_table['grid_'+str(n)+'_calculated'][index] for n in [1,2,3,4,5,6,7]])==0)[0][0]
            index_missing_grid = np.where(progress_table['grid_1'] == progress_table['grid_'+str(missing_grid+1)][index])[0]
            if len(index_missing_grid) > 0:
                index_missing_grid = index_missing_grid[0]
            else:
                index_missing_grid = -1
            progress_table['next_calculation'][index] = index_missing_grid
        if progress_table['grids_calculated'][index] == 2:
            missing_grid = np.where(np.array([progress_table['grid_'+str(n)+'_calculated'][index] for n in [1,2,3,4,5,6,7]])==0)[0][0]
            index_missing_grid = np.where(progress_table['grid_1'] == progress_table['grid_'+str(missing_grid+1)][index])[0]
            if len(index_missing_grid) > 0:
                index_missing_grid = index_missing_grid[0]
            else:
                index_missing_grid = -1
            progress_table['next_calculation'][index] = index_missing_grid
        if progress_table['grids_calculated'][index] == 1:
            missing_grid = np.where(np.array([progress_table['grid_'+str(n)+'_calculated'][index] for n in [1,2,3,4,5,6,7]])==0)[0][0]
            index_missing_grid = np.where(progress_table['grid_1'] == progress_table['grid_'+str(missing_grid+1)][index])[0]
            if len(index_missing_grid) > 0:
                index_missing_grid = index_missing_grid[0]
            else:
                index_missing_grid = -1
            progress_table['next_calculation'][index] = index_missing_grid

    for index in range(len(progress_table['grids_trainingset'])):
        if progress_table['grids_trainingset'][index] == 6:
            missing_grid = np.where(np.array([progress_table['grid_'+str(n)+'_trainingset'][index] for n in [1,2,3,4,5,6,7]])==0)[0][0]
            index_missing_grid = np.where(progress_table['grid_1'] == progress_table['grid_'+str(missing_grid+1)][index])[0]
            if len(index_missing_grid) > 0:
                index_missing_grid = index_missing_grid[0]
            else:
                index_missing_grid = -1
            progress_table['next_conversion'][index] = index_missing_grid
        if progress_table['grids_trainingset'][index] == 5:
            missing_grid = np.where(np.array([progress_table['grid_'+str(n)+'_trainingset'][index] for n in [1,2,3,4,5,6,7]])==0)[0][0]
            index_missing_grid = np.where(progress_table['grid_1'] == progress_table['grid_'+str(missing_grid+1)][index])[0]
            if len(index_missing_grid) > 0:
                index_missing_grid = index_missing_grid[0]
            else:
                index_missing_grid = -1
            progress_table['next_conversion'][index] = index_missing_grid
        if progress_table['grids_trainingset'][index] == 4:
            missing_grid = np.where(np.array([progress_table['grid_'+str(n)+'_trainingset'][index] for n in [1,2,3,4,5,6,7]])==0)[0][0]
            index_missing_grid = np.where(progress_table['grid_1'] == progress_table['grid_'+str(missing_grid+1)][index])[0]
            if len(index_missing_grid) > 0:
                index_missing_grid = index_missing_grid[0]
            else:
                index_missing_grid = -1
            progress_table['next_conversion'][index] = index_missing_grid

    return(progress_table)
progress_table = get_progress()
x=90
progress_table[np.where((progress_table['grids_calculated'] >= 6) & (progress_table['trainingset_done'] != 1))[0]][x:15+x]

/var/folders/vz/3qk7_1lx4vq905d6y0hspntr0000gs/T/ipykernel_30570/150773874.py:100: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  progress_table['grids_calculated'] = np.sum(progress_table['grid_'+str(n)+'_calculated'] for n in [1,2,3,4,5,6,7])
/var/folders/vz/3qk7_1lx4vq905d6y0hspntr0000gs/T/ipykernel_30570/150773874.py:101: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  progress_table['grids_trainingset'] = np.sum(progress_table['grid_'+str(n)+'_trainingset'] for n in [1,2,3,4,5,6,7])


index_subgrid,trainingset_done,grids_trainingset,grids_calculated,next_conversion,next_calculation,grid_1,grid_2,grid_3,grid_4,grid_5,grid_6,grid_7,grid_1_calculated,grid_2_calculated,grid_3_calculated,grid_4_calculated,grid_5_calculated,grid_6_calculated,grid_7_calculated,grid_1_trainingset,grid_2_trainingset,grid_3_trainingset,grid_4_trainingset,grid_5_trainingset,grid_6_trainingset,grid_7_trainingset
int64,int64,float64,float64,int64,int64,str15,str15,str15,str16,str15,str15,str15,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


In [70]:
print(len(np.unique(list(progress_table['next_calculation'][np.where((progress_table['grids_calculated'] >= 6) & (progress_table['trainingset_done'] != 1))[0]]))))
np.unique(list(progress_table['next_calculation'][np.where((progress_table['grids_calculated'] >= 6) & (progress_table['trainingset_done'] != 1))[0]]))

0


array([], dtype=float64)

In [71]:
progress_table[np.where((progress_table['grids_calculated'] == 7) & (progress_table['trainingset_done'] != 1))[0]]

index_subgrid,trainingset_done,grids_trainingset,grids_calculated,next_conversion,next_calculation,grid_1,grid_2,grid_3,grid_4,grid_5,grid_6,grid_7,grid_1_calculated,grid_2_calculated,grid_3_calculated,grid_4_calculated,grid_5_calculated,grid_6_calculated,grid_7_calculated,grid_1_trainingset,grid_2_trainingset,grid_3_trainingset,grid_4_trainingset,grid_5_trainingset,grid_6_trainingset,grid_7_trainingset
int64,int64,float64,float64,int64,int64,str15,str15,str15,str16,str15,str15,str15,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


In [72]:
todo = [str(x) for x in progress_table['index_subgrid'][np.where(((progress_table['next_conversion'] == -1) | (progress_table['next_conversion'] == 0)) & (progress_table['grids_calculated'] == 7) & (progress_table['trainingset_done'] != 1))[0]]]
todo1 = todo[:int(len(todo)/2)]
todo2 = todo[int(len(todo)/2):]

print(','.join(todo))
for t in [todo1,todo2]:
    print('import os')
    print('ps = ['+','.join(t)+']')
    print('for p in ps:')
    print("    os.system('ipython galah_dr4_thepayne_grid_interpolation_training_extra6.py '+str(p))")



import os
ps = []
for p in ps:
    os.system('ipython galah_dr4_thepayne_grid_interpolation_training_extra6.py '+str(p))
import os
ps = []
for p in ps:
    os.system('ipython galah_dr4_thepayne_grid_interpolation_training_extra6.py '+str(p))


In [81]:
upcoming_priority = priority[[x not in trained_6 for x in priority['grid_index']]]
upcoming_priority[0]

grid_index,counts_dr4_init
int64,int64
2002,35638
